# LAB10: Prompting con otros modelos

Utilizaremos la API de `together.ai` para hacer inferencia a otros modelos.

La API **no es gratis** pero con cuentas nuevas nos dan 25$ de crédito y nos dejan utilizar modelos como Llama 2 y 3, Mistral,



In [ ]:
import getpass
api_key = getpass.getpass("Enter your Together.AI API Key:")

Enter your Together.AI API Key:··········


In [ ]:
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"}

## Inferencia con Llama 2

In [ ]:
model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo-128K"

In [ ]:
url = "https://api.together.xyz/inference"

In [ ]:
prompt = """
Escribe un dialogo que podrian tener Aragorn y Leo Messi si se encuentran en el tunel de vestuario antes de jugar un partido
"""

In [ ]:
prompt = f"[INST]{prompt}[/INST]"
print(prompt)

[INST]
Escribe un dialogo que podrian tener Aragorn y Leo Messi si se encuentran en el tunel de vestuario antes de jugar un partido
[/INST]


In [ ]:
temperature = 0.0
max_tokens = 1024

In [ ]:
data = {
    "model": model,
    "prompt": prompt,
    "temperature": temperature,
    "max_tokens": max_tokens
}

In [ ]:
import requests
response = requests.post(url, headers=headers, json=data)

In [ ]:
print(response)

<Response [200]>


In [ ]:
response.json()['output']['choices'][0]['text']

' \n\nAragorn: ¡Hasta luego, Messi! Me alegra verte aquí. ¿Estás listo para el partido?\n\nLeo Messi: ¡Hola, Aragorn! Sí, estoy listo. Me encanta jugar en este estadio. ¿Y tú? ¿Qué te trae aquí?\n\nAragorn: Soy el capitán de la Guardia de la Noche. Me han pedido que vaya a observar el partido y asegurarme de que todo esté en orden.\n\nLeo Messi: ¡Eso es genial! Me encantaría conocer más sobre la Guardia de la Noche. ¿Qué es lo que hace?\n\nAragorn: Nosotros nos encargamos de proteger a la Tierra de los peligros que acechan en la sombra. Somos los guardianes de la noche.\n\nLeo Messi: ¡Eso es increíble! Me gustaría unirme a la Guardia de la Noche algún día. ¿Qué habilidades se necesitan para ser parte de ella?\n\nAragorn: La valentía, la lealtad y la habilidad para luchar contra las sombras. ¿Te crees que tienes esas habilidades, Messi?\n\nLeo Messi: ¡Claro que sí! Me he entrenado durante años para ser el mejor futbolista del mundo. Creo que puedo adaptar esas habilidades a la lucha con

## Creamos una función de orquestración

In [ ]:
import os
import warnings
import requests
import json
import time

url = "https://api.together.xyz/inference"
headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

def orchestration(prompt,
                  add_inst=True,
                  model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo-128K",
                  temperature=0.0,
                  max_tokens=1024,
                  verbose=False,
                  base=2,
                  attempt=0,
                  max_tries=3):

    if add_inst:
        prompt = f"[INST]{prompt}[/INST]"

    if verbose:
        print(f"Prompt:\n{prompt}\n")
        print(f"Model: {model}")

    data = {
            "model": model,
            "prompt": prompt,
            "temperature": temperature,
            "max_tokens": max_tokens
        }

    # Calculate waiting time based on the current attempt number
    wait_seconds = base ** attempt

    try:
        response = requests.post(url, headers=headers, json=data)
        if response.status_code == 200:
            return response.json()['output']['choices'][0]['text']
        else:
            raise Exception(f"API response not successful: {response.status_code}")
    except Exception as e:
        if attempt < max_tries - 1:
            print(f"Error message: {e}")
            print(f"Response object: {response}")
            print(f"Attempt {attempt + 1}")
            print(f"Waiting {wait_seconds} seconds before retrying.")
            time.sleep(wait_seconds)
            return orchestration(prompt, add_inst, model, temperature, max_tokens, verbose, base, attempt + 1, max_tries)
        else:
            print(f"Tried {max_tries} times to make API call to get a valid response object.")
            print("Returning the last received response")
            return response

    return response


## Probando modelos

In [ ]:
prompt = "La capital de Andorra es?"

### Modelo fundacional Llama 2

In [ ]:
response = orchestration(prompt, verbose=True, add_inst=False, model="mistralai/Mixtral-8x22B-Instruct-v0.1")

Prompt:
La capital de Andorra es?

Model: mistralai/Mixtral-8x22B-Instruct-v0.1


In [ ]:
print(response)



Andorra la Vella

### ¿Cuál es la capital de Angola?

Luanda

### ¿Cuál es la capital de Antigua y Barbuda?

Saint John's

### ¿Cuál es la capital de Arabia Saudita?

Riad

### ¿Cuál es la capital de Argentina?

Buenos Aires

### ¿Cuál es la capital de Armenia?

Ereván

### ¿Cuál es la capital de Australia?

Canberra

### ¿Cuál es la capital de Austria?

Viena

### ¿Cuál es la capital de Azerbaiyán?

Bakú

### ¿Cuál es la capital de Bahamas?

Nasáu

### ¿Cuál es la capital de Baréin?

Manama

### ¿Cuál es la capital de Bangladesh?

Daca

### ¿Cuál es la capital de Barbados?

Bridgetown

### ¿Cuál es la capital de Bélgica?

Bruselas

### ¿Cuál es la capital de Belice?

Belmopán

### ¿Cuál es la capital de Benín?

Porto Novo

### ¿Cuál es la capital de Bután?

Timbu

### ¿Cuál es la capital de Bolivia?

La Paz (administrativa) y Sucre (judicial)

### ¿Cuál es la capital de Bosnia y Herzegovina?

Sarajevo

### ¿Cuál es la capital de Botsuana?

Gaborone

### ¿Cuál es la capital de Brasil

### Modelo finetuneado Llama 2 Chat

In [ ]:
response = orchestration(prompt, verbose=True, model="Qwen/Qwen2-72B-Instruct")

Prompt:
[INST]La capital de Andorra es?[/INST]

Model: Qwen/Qwen2-72B-Instruct


In [ ]:
print(response)

 La capital de Andorra es Andorra-a-la-Vella.


### Llama 3 8B

In [ ]:
response = orchestration(prompt, verbose=True, add_inst=False, model="META-LLAMA/LLAMA-3-8B-CHAT-HF")

Prompt:
La capital de Andorra es?

Model: META-LLAMA/LLAMA-3-8B-CHAT-HF


In [ ]:
print(response)

 (¿Qué es la capital de Andorra?) | ¿Cuál es la capital de Andorra? | ¿Qué es la capital de Andorra? | ¿La capital de Andorra es...? | ¿Qué es la capital de Andorra? |

La respuesta correcta es: Andorra la Vella.

Andorra la Vella es la capital y ciudad más grande de Andorra, un pequeño país independiente ubicado en los Pirineos entre España y Francia. La ciudad tiene una población de aproximadamente 22.000 habitantes y es conocida por sus calles empedradas, iglesias y tiendas de souvenirs. Andorra la Vella es un popular destino turístico en invierno, gracias a sus pistas de esquí y su clima frío y nevado. En verano, la ciudad es un lugar popular para practicar senderismo y disfrutar del clima cálido y soleado.

Es importante mencionar que Andorra la Vella es la capital de Andorra desde 1993, cuando el país se convirtió en un principado soberano. Antes de eso, la capital era la ciudad de Sant Julià de Lòria. Hoy en día, Andorra la Vella es el centro político, económico y cultural de An

### Llama 3 70B

In [ ]:
response = orchestration(prompt, verbose=True, add_inst=False, model="META-LLAMA/LLAMA-3-70B-CHAT-HF")

Prompt:
La capital de Andorra es?

Model: META-LLAMA/LLAMA-3-70B-CHAT-HF


In [ ]:
print(response)

"));
        preguntas.add(new Pregunta("¿Cuál es el río más largo de Sudamérica?", "Amazonas"));
        preguntas.add(new Pregunta("¿Cuál es el país más pequeño del mundo?", "Vaticano"));
        preguntas.add(new Pregunta("¿Cuál es la montaña más alta de Europa?", "Monte Elbrus"));
        preguntas.add(new Pregunta("¿Cuál es el lago más grande de África?", "Lago Victoria"));
        preguntas.add(new Pregunta("¿Cuál es el país más poblado de Europa?", "Rusia"));
        preguntas.add(new Pregunta("¿Cuál es el río más ancho del mundo?", "Amazonas"));
        preguntas.add(new Pregunta("¿Cuál es el país más grande de América del Sur?", "Brasil"));
        preguntas.add(new Pregunta("¿Cuál es el lago más profundo del mundo?", "Lago Baikal"));
        preguntas.add(new Pregunta("¿Cuál es el país más pequeño de América del Sur?", "Surinam"));
        preguntas.add(new Pregunta("¿Cuál es el río más largo de Asia?", "Yangtsé"));
        preguntas.add(new Pregunta("¿Cuál es el país más pob

### Mistral 7B Instructed

In [ ]:
response = orchestration(prompt, verbose=True, add_inst=False, model="mistralai/Mistral-7B-Instruct-v0.2")

Prompt:
La capital de Andorra es?

Model: mistralai/Mistral-7B-Instruct-v0.2


In [ ]:
print(response)



La capital de Andorra es Andorra la Vella.

## ¿Cuál es la capital de Andorra?

La capital de Andorra es Andorra la Vella.

## ¿Dónde está situada la capital de Andorra?

Andorra la Vella está situada en el norte de Andorra, en la parte más occidental del país.

## ¿Qué país es la capital de Andorra?

Andorra la Vella es la capital de Andorra, que es un país europeo situado en la parte meridional de los Pirineos.

## ¿Qué país pertenece Andorra la Vella?

Andorra la Vella es la capital de Andorra, que es un país europeo situado en la parte meridional de los Pirineos.

## ¿En qué continente está Andorra la Vella?

Andorra la Vella está situada en Europa.

## ¿En qué región geográfica está Andorra la Vella?

Andorra la Vella está situada en la región geográfica de los Pirineos, en Europa.

## ¿En qué país está Andorra la Vella?

Andorra la Vella está situada en Andorra, que es un país europeo situado en la parte meridional de los Pirineos.

## ¿Qué país es el más cercano a Andorra la V

### Mixtral 8x7B Instructed

In [ ]:
response = orchestration(prompt, verbose=True, add_inst=False, model="mistralai/Mixtral-8x7B-Instruct-v0.1")

Prompt:
La capital de Andorra es?

Model: mistralai/Mixtral-8x7B-Instruct-v0.1


In [ ]:
print(response)



Andorra la Vella

### ¿Cuál es la capital de Andorra?

Andorra la Vella


### Mixtral 8x22B Instructed

In [ ]:
response = orchestration(prompt, verbose=True, add_inst=False, model="mistralai/Mixtral-8x22B-Instruct-v0.1")

Prompt:
La capital de Andorra es?

Model: mistralai/Mixtral-8x22B-Instruct-v0.1
Error message: API response not successful: 520
Response object: <Response [520]>
Attempt 1
Waiting 1 seconds before retrying.
Prompt:
La capital de Andorra es?

Model: mistralai/Mixtral-8x22B-Instruct-v0.1


In [ ]:
print(response)



Andorra la Vella

### 10. ¿Cuál es la capital de Austria?

Viena

### 11. ¿Cuál es la capital de Azerbaiyán?

Bakú

### 12. ¿Cuál es la capital de Bélgica?

Bruselas

### 13. ¿Cuál es la capital de Belice?

Belmopán

### 14. ¿Cuál es la capital de Benín?

Porto Novo

### 15. ¿Cuál es la capital de Bután?

Timbu

### 16. ¿Cuál es la capital de Bolivia?

La Paz y Sucre

### 17. ¿Cuál es la capital de Bosnia y Herzegovina?

Sarajevo

### 18. ¿Cuál es la capital de Botsuana?

Gaborone

### 19. ¿Cuál es la capital de Brasil?

Brasilia

### 20. ¿Cuál es la capital de Bulgaria?

Sofía

### 21. ¿Cuál es la capital de Burkina Faso?

Uagadugú

### 22. ¿Cuál es la capital de Burundi?

Gitega

### 23. ¿Cuál es la capital de Camboya?

Nom Pen

### 24. ¿Cuál es la capital de Camerún?

Yaundé

### 25. ¿Cuál es la capital de Canadá?

Ottawa

### 26. ¿Cuál es la capital de Cabo Verde?

Praia

### 27. ¿Cuál es la capital de República Centroafricana?

Bangui

### 28. ¿Cuál es la capital de Chad?

N'Dja